In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window


### **Data Reading**

In [0]:
df = spark.read.format("parquet").load("abfss://bronze@databricksetecharif.dfs.core.windows.net/orders")

In [0]:
df.display()

In [0]:
df.drop("_rescued_data").display()


In [0]:
df.display()

In [0]:
df.printSchema

In [0]:
df.withColumnRenamed("_rescued_data","rescued_data").display()

In [0]:
df = df.drop("_rescued_data")
df.display()

In [0]:
df = df.withColumn("order_date",to_timestamp(col('order_date')))

In [0]:
df.display()

In [0]:
df = df.withColumn('year',year(col("order_date")))

In [0]:
df.display()

In [0]:
windowspec = Window.partitionBy(col("year")).orderBy(desc("total_amount"))
df1= df.withColumn("flag",rank().over(windowspec))
df1.display()

In [0]:
df2= df1.withColumn("rank_flag",dense_rank().over(windowspec))
df2.display()

In [0]:
df3= df2.withColumn("row_number",row_number().over(windowspec))
df3.display()

### **Classes - OOP**

In [0]:
class windows:
    def dense_rank(self,df,windowspec):
        return df.withColumn("rank_flag",dense_rank().over(windowspec))
    def rank(self,df):
        return df.withColumn("flag",rank().over(windowspec))
    def row_number(self,df):
        return df.withColumn("row_number",row_number().over(windowspec))

In [0]:
df.display()

In [0]:
obj = windows()
df5 = obj.dense_rank(df,windowspec)

In [0]:
df5.display()

In [0]:
df.display()

In [0]:
df_new = df.drop("year")

In [0]:
df_new.display()

In [0]:
df.write.format("delta").mode("overwrite").save("abfss://silver@databricksetecharif.dfs.core.windows.net/orders")

In [0]:
%sql
CREATE TABLE IF NOT EXISTS databricks_catalog.silver.orders_silver
USING DELTA
LOCATION 'abfss://silver@databricksetecharif.dfs.core.windows.net/orders'

In [0]:
%sql
select * from databricks_catalog.silver.orders_silver